### RNN for recognizing genetic code

#### Task
In this challenge, we to train a classifier for sequences of genetic code.
Each sequence is represented by a string of letters [‘A’, ‘C’, ‘G’, ’T’] and belongs to one of five categories/classes labelled [0,…,4].

For training purposes, you will find 400 labelled sequences, each of length 400 characters (sequences: data_x, labels: data_y).

To validate your model, you have a further 100 labelled sequences (val_x, val_y) with 1200 characters each.

Finally, you have 250 unlabelled sequences (test_x, 2000 characters) which need to be classified.

Hint: Training recurrent networks is very expensive! Do not start working on this challenge to late or you will not manage to finish in time.

#### Approach

We will use LSTM units in a recurrent NN primarily based on this paper -> https://arxiv.org/pdf/1608.03644.pdf .
The architecture is a combination of convolutional layers and recurrent LSTM layers.
One can clearly see in this paper, that CNNs outperform RNNs with LSTM units in this task, but we are using RNNs for the sake of learning here.

## Imports

In [1]:
import numpy as np

import keras
from keras import backend as K
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Reshape, Flatten, \
                         Conv2D, MaxPooling2D, BatchNormalization, Bidirectional
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

# from sklearn.preprocessing import LabelEncoder

K.set_image_data_format("channels_last")

Using TensorFlow backend.


### Load Data

In [2]:
with np.load('./data/rnn-challenge-data.npz') as f:
    train_x = f['data_x']
    train_y = f['data_y']
    val_x = f['val_x']
    val_y = f['val_y']
    test_x = f['test_x']
    
print('train_x shape: {}'.format(train_x.shape))
print('train_y shape: {}'.format(train_y.shape))
print('val_x shape: {}'.format(val_x.shape))
print('val_y shape: {}'.format(val_y.shape))
print('test_x shape: {}'.format(test_x.shape))

train_x shape: (400,)
train_y shape: (400,)
val_x shape: (100,)
val_y shape: (100,)
test_x shape: (250,)


### Let's see what's inside

In [3]:
print('X: {}'.format(train_x[0]))
print('y: {}'.format(train_y[0]))

X: CTAGCTGAGCTACTGAGCTACAGTTGACTGACCAGTCAGTGCTAGCTACTGACAGTCTGACAGTTGACCTGACTGATGACCAGTCTAGCAGTGCTACTAGCTAGGCTACAGTCAGTTGACCAGTCTGACAGTCAGTCTGACTGACAGTCAGTCTAGGCTATGACCTGACTGATGACCTGACTGACTGACAGTCTGACTGATGACGCTATGACCTGACTAGCTAGCAGTTGACTGACCTGACAGTGCTACTAGCAGTTGACCAGTGCTACAGTCTGATGACTGACCTGACAGTCTAGGCTACAGTTGACCTGACAGTCAGTGCTACTGACAGTCTAGTGACCAGTCAGTCAGTTGACCTGACTAGCAGTTGACGCTATGACCAGTCTGACAGTGCTACTAG
y: 2


### DNA Code encoding

In [4]:
representations = ['A', 'C', 'G', 'T']
max_sequence_len = 2000
reps_len = len(representations)
eye = np.eye(reps_len)

def one_hot(X):
    encoded = []
    
    for code in X:
#         encoded.append([representations.index(c) for c in code])
        encoded.append(np.vstack([
            np.array([eye[representations.index(c)] for c in code]),
            np.zeros((max_sequence_len - len(code), reps_len))
        ]
        ))

        # TODO: do a better padding
    
    # because of the way LSTM is implemented in keras, we need the columns to be our data points
    # and in the rows we need all the features
    return np.array(encoded).swapaxes(1,2)

try:
    X_train = one_hot(train_x)
    y_train = np.eye(5)[train_y]
    X_val = one_hot(val_x)
    y_val = np.eye(5)[val_y]
    X_test = one_hot(test_x)

    del train_x, train_y, val_x, val_y, test_x
except Exception as e:
    print(e)
    pass
    # step already done
    
X_train.shape

(400, 4, 2000)

## Hyperparams

In [5]:
embedding_size = 100
sequence_len = X_train.shape[1]
embedding_image_shape = tuple(np.array([np.sqrt(embedding_size * sequence_len)] * 2, dtype=int))

conv_kernel = (9,9)
conv_stride = (1,1)
conv_filters = 128
conv_padding = 'same'

# pool_size = (7,7)
# pool_stride = (2,2)
# pool_padding = 'valid'

lstm_layers = 32

optimizer = Adam()
batch_size = 32
epochs = 150

labels_count = 5

## Define Model

In [6]:
model_input = Input(shape=X_train.shape[1:])
x = model_input
x = Reshape((*X_train.shape[1:], 1))(x)
x = Conv2D(conv_filters, kernel_size=conv_kernel, padding=conv_padding, strides=conv_stride, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Conv2D(1, kernel_size=conv_kernel, padding=conv_padding, strides=conv_stride, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Reshape(X_train.shape[1:])(x)
# TODO: try merge mode
x = Bidirectional(LSTM(lstm_layers, dropout=0.5, recurrent_dropout=0.2))(x)
x = Dense(labels_count, activation='softmax')(x)

model = Model(model_input, x, name='model')
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 2000)           0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 2000, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 2000, 128)      10496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 2000, 128)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 2000, 128)      512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 2000, 1)        10369     
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 2000, 1)        0         
__________

### Train

In [7]:
checkpointer = ModelCheckpoint(filepath='./model/weights.hdf5', verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir='./stats/dim_reduction')
print(X_train.shape)
stop = EarlyStopping(monitor="val_loss", min_delta=0.0001, patience=25, mode="min")
history = model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_data=(X_val, y_val),
                callbacks=[checkpointer, tensorboard, stop])

(400, 4, 2000)
Train on 400 samples, validate on 100 samples
Epoch 1/150
400/400 [==============================] - 92s 230ms/step - loss: 1.6477 - categorical_accuracy: 0.2225 - val_loss: 1.6437 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.64374, saving model to ./model/weights.hdf5
Epoch 2/150
400/400 [==============================] - 90s 225ms/step - loss: 1.5147 - categorical_accuracy: 0.3300 - val_loss: 1.6995 - val_categorical_accuracy: 0.2400

Epoch 00002: val_loss did not improve from 1.64374
Epoch 3/150
400/400 [==============================] - 87s 218ms/step - loss: 1.1927 - categorical_accuracy: 0.5925 - val_loss: 1.5069 - val_categorical_accuracy: 0.2600

Epoch 00003: val_loss improved from 1.64374 to 1.50687, saving model to ./model/weights.hdf5
Epoch 4/150
400/400 [==============================] - 88s 220ms/step - loss: 0.7824 - categorical_accuracy: 0.7800 - val_loss: 1.0818 - val_categorical_accuracy: 0.4900

Epoch 00004: val_loss 


Epoch 00034: val_loss did not improve from 0.21553
Epoch 35/150
400/400 [==============================] - 84s 211ms/step - loss: 0.0024 - categorical_accuracy: 1.0000 - val_loss: 0.2534 - val_categorical_accuracy: 0.9800

Epoch 00035: val_loss did not improve from 0.21553
Epoch 36/150
400/400 [==============================] - 84s 210ms/step - loss: 0.0023 - categorical_accuracy: 1.0000 - val_loss: 0.2629 - val_categorical_accuracy: 0.9700

Epoch 00036: val_loss did not improve from 0.21553
Epoch 37/150
400/400 [==============================] - 84s 210ms/step - loss: 0.0021 - categorical_accuracy: 1.0000 - val_loss: 0.2566 - val_categorical_accuracy: 0.9800

Epoch 00037: val_loss did not improve from 0.21553
Epoch 38/150
400/400 [==============================] - 84s 211ms/step - loss: 0.0022 - categorical_accuracy: 1.0000 - val_loss: 0.2446 - val_categorical_accuracy: 0.9900

Epoch 00038: val_loss did not improve from 0.21553
Epoch 39/150
400/400 [==============================] - 

In [8]:
model.load_weights('./model/weights.hdf5')
loss, acc = model.evaluate(X_val, y_val)
print('Accuracy: {}'.format(acc))

100/100 [==============================] - 5s 50ms/step
Accuracy: 0.99


### Predictions and check confidence

In [10]:
one_hot_predictions = model.predict(X_test)
predictions = np.argmax(one_hot_predictions, axis=1)
print(one_hot_predictions[:5])
print(predictions[:5])

[[0.04641771 0.02749521 0.7219298  0.1031497  0.1010076 ]
 [0.03334371 0.04376873 0.62636894 0.03993057 0.25658798]
 [0.02275329 0.18065219 0.35334268 0.4238008  0.01945102]
 [0.03114861 0.1328021  0.48470274 0.33025292 0.02109354]
 [0.1473253  0.0177479  0.66418207 0.06035194 0.11039281]]
[2 2 3 2 2]


In [12]:
np.save('prediction.npy', predictions)